# Langchain Testdrive

This notebooks are a series of testdrives with the [langchain](https://python.langchain.com/) Framework. I start off with the ones from the [Quickstart](https://python.langchain.com/docs/get_started/quickstart.html) section and build up from there to something interesting - wherever my head takes me.

## Installing LangChain and OpenAI

Install langchain with a simple `pip install`

In [ ]:
pip install langchain

Langchain needs access to LLM Provider. I am using OpenAI here.

In [ ]:
pip install openai

Accessing OpenAI requires an API key. I have created an account there first. On the [API keys](https://platform.openai.com/account/api-keys) page on OpenAI, you can create an API key. Set the API Key as an environment variable `OPENAI_API_KEY`:

In [5]:
%%bash 
export OPENAI_API_KEY="..."

Let's get the API key from the user directly

In [7]:
from getpass import getpass
OPENAI_API_KEY = getpass('OpenAI API Key')

## Predictions from OpenAI with LangChain

LangChain's basic building block is the LLM. Here we use OpenAI to provide some texts on a given Prompt.

First we setup the llm to be OpenAI and we set the temperature to a high value, so that the responses are more random.

In [8]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9, 
             openai_api_key=OPENAI_API_KEY) #You can set the API key here as well

Now let us create a prediction:

In [9]:
prompt = "What would be a good name for a dog coming from Frankfurt?"
llm.predict(prompt)

'\n\nFrankfurter.'

## Building Name-A-Pet GPT - Prompt Templates and Chains

Now we have the building blocks for building a service, where a user could get good suggestions for a pet name. However we direct the user and let him not bother him on getting the right prompt for the LLM. This is where we use `PromptTemplate`:

In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What would be a good name for a {pet} coming from {location}?")
prompt.format(pet="cat", location="Darmstadt")

'What would be a good name for a cat coming from Darmstadt?'

We have the LLM and the prompt template. To combine the two to get a prediction, we chain the two building blocks.

In [11]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.run(pet="cat", location="Darmstadt")

'\n\nDarmy the Cat.'

Great! Now wrap this in an API and build a WebApp in front of it and voilá you have a web service for generating pet names.

# Building a Stock Idea Generator

***This is just a fun project. Please do your own research when it comes to investing or consult a registered financial advisor. Whatever the code is generating below, is not an investment advise***

I like investing in Stocks and coming up for new ideas for investing could be a very interesting use case.

For this we want to create a more complex chain with LangChain. The idea is to use the `Agent` to create a conversational ChatBot. The agent will use Google Search to get more context on the prompts provided and use them in the LLM to provide up to date and hopefully very interesting results!

## Importing Libraries

First step is to install some more libraries. Here we get the `google-search-results` python package. It uses the [SerpApi](serpapi.com). 

In [ ]:
pip install google-search-results

## Building the App

Import the necessary libraries.

In [12]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

Get yourself registered on [SerpApi](serpapi.com). There is a free subscription available that allows a few requests on their service per month. On [SerpApi's API Key page](https://serpapi.com/manage-api-key) you will get the `SERP_API_KEY`. 

In [14]:
SERP_API_KEY = getpass('Serp API Key')

Tools are integrations to other services in the LangChain that provide the LLM with more context. Here we are using Google search results. When defining a tool, a `description` is important. The LLM model in LangChain decides according to this description, whether the tool will be used.

In [23]:

search = SerpAPIWrapper(serpapi_api_key=SERP_API_KEY)
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world or latest information on companies and stocks"
    ),
]

To keep the context within the conversation, the `ConversaionBufferMemory` is used.

In [16]:
memory = ConversationBufferMemory(memory_key="chat_history")

Just initialize the LLM, the tools and memory and the agent is done.

In [18]:
llm=OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
agent_chain = initialize_agent(tools=tools, 
                               llm=llm, 
                               agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, 
                               memory=memory)

Provide the Prompts from the user and get the conversation going!

In [20]:
agent_chain.run(input=
                """I am a risk-averse investor and I am interested in stocks in EMEA. 
                Can you list me 10 stocks with growth potential? 
                Please ensure that the companies listed are the best-in-class in sustainability.
                """)



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: 10 stocks with growth potential in EMEA with best-in-class sustainability
Observation: 10 Best Stocks for Environmental, Social, and Governance (ESG) Investing · 10 best ESG stocks right now · 1. Nvidia · 2. Microsoft · 3. Best Buy · 4. Adobe · 5. Pool.
Thought: Do I need to use a tool? No
AI: Here are the 10 stocks with growth potential in EMEA that have best-in-class sustainability: Nvidia, Microsoft, Best Buy, Adobe, Pool, Apple, Amazon, Alphabet, Johnson & Johnson, and Unilever. I hope this helps you make an informed decision.

> Finished chain.


'Here are the 10 stocks with growth potential in EMEA that have best-in-class sustainability: Nvidia, Microsoft, Best Buy, Adobe, Pool, Apple, Amazon, Alphabet, Johnson & Johnson, and Unilever. I hope this helps you make an informed decision.'

In [22]:
agent_chain.run(input=
                """Limit the list to 3 stocks that are from EMEA.
                """)



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Current Search
Action Input: List 3 stocks from EMEA with growth potential and best-in-class sustainability
Observation: Put a little green in your wallet by investing in these growing areas. ... Eco-friendly investments can provide profits as well as environmental benefits.
Thought: Do I need to use a tool? No
AI: The three stocks from EMEA with growth potential and best-in-class sustainability are Nvidia, Microsoft, and Unilever. Investing in these stocks can provide both financial and environmental benefits.

> Finished chain.


'The three stocks from EMEA with growth potential and best-in-class sustainability are Nvidia, Microsoft, and Unilever. Investing in these stocks can provide both financial and environmental benefits.'

Ok, Nvidia, Microsoft are US based companies. But as we see, how easy it is to build specialized AI solutions with LLMs and then LangChain framework, there is no wonder that Nvidia and Microsoft are rated to highly by this tool!